In [1]:
import numpy as np
from scipy.stats import t
from scipy.stats import norm
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd
import io
import requests
import seaborn as sns
import statistics
import datetime
from datetime import datetime

%matplotlib inline

Представленный датасет - база заказов онлайн магазина из Великобритании.

Значения столбцов:
- InvoiceNo - Номер счет-фактуры;
- StockCode - инвентарный номер;
- Description - описание;
- Quantity - количество;
- InvoiceDate - дата счета;
- UnitPrice - цена за единицу товара;
- CustomerID - пользовательский идентификатор;
- Country - страна

In [3]:
data = pd.read_csv("Data_Sample_(AIC).csv")
data.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,"3,39",17850.0,United Kingdom
1,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/1/10 8:34,"1,69",13047.0,United Kingdom
2,536367,21755,LOVE BUILDING BLOCK WORD,3,12/1/10 8:34,"5,95",13047.0,United Kingdom
3,536368,22914,BLUE COAT RACK PARIS FASHION,3,12/1/10 8:34,"4,95",13047.0,United Kingdom
4,536373,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 9:02,"2,75",17850.0,United Kingdom
5,536375,82494L,WOODEN FRAME ANTIQUE WHITE,6,12/1/10 9:32,"2,55",17850.0,United Kingdom
6,536378,21033,JUMBO BAG CHARLIE AND LOLA TOYS,10,12/1/10 9:37,"2,95",14688.0,United Kingdom
7,536381,82567,"AIRLINE LOUNGE,METAL SIGN",2,12/1/10 9:41,"2,1",15311.0,United Kingdom
8,536381,21169,YOU'RE CONFUSING ME METAL SIGN,3,12/1/10 9:41,"1,69",15311.0,United Kingdom
9,536381,21175,GIN + TONIC DIET METAL SIGN,2,12/1/10 9:41,"2,1",15311.0,United Kingdom


   Отчет на основе данного датасета может быть полезен коммерческому директору, маркетологу. 
   - Целью которых может быть - максимизация прибыли. Увеличение доли рынка.
   Проанализируем имеющиеся данные

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54214 entries, 0 to 54213
Data columns (total 8 columns):
InvoiceNo      54214 non-null object
StockCode      54214 non-null object
Description    54080 non-null object
Quantity       54214 non-null int64
InvoiceDate    54214 non-null object
UnitPrice      54214 non-null object
CustomerID     40643 non-null float64
Country        54214 non-null object
dtypes: float64(1), int64(1), object(6)
memory usage: 3.3+ MB


  Дата сет содержит 54214 строк. Как мы можем увидеть ниже датасет содержит 134 пустых значения в столбце Description и 13571 пустое значение в столбце CustomerID. Для дальнейшего анализа нам необходимо избавиться от null значений.

In [5]:
data.isnull().sum()

InvoiceNo          0
StockCode          0
Description      134
Quantity           0
InvoiceDate        0
UnitPrice          0
CustomerID     13571
Country            0
dtype: int64

In [6]:
data = data.dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40643 entries, 0 to 54213
Data columns (total 8 columns):
InvoiceNo      40643 non-null object
StockCode      40643 non-null object
Description    40643 non-null object
Quantity       40643 non-null int64
InvoiceDate    40643 non-null object
UnitPrice      40643 non-null object
CustomerID     40643 non-null float64
Country        40643 non-null object
dtypes: float64(1), int64(1), object(6)
memory usage: 2.8+ MB


In [6]:
# data.to_csv('Final_Work.csv', sep='\t')

В нашем датасете столбец UnitPrice является объектом, переведем значения данного столбца в числа

In [7]:
data['UnitPrice'] = data['UnitPrice'].apply(lambda x: x.replace(',', '.'))
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850.0,United Kingdom
1,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/1/10 8:34,1.69,13047.0,United Kingdom
2,536367,21755,LOVE BUILDING BLOCK WORD,3,12/1/10 8:34,5.95,13047.0,United Kingdom
3,536368,22914,BLUE COAT RACK PARIS FASHION,3,12/1/10 8:34,4.95,13047.0,United Kingdom
4,536373,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 9:02,2.75,17850.0,United Kingdom


In [8]:
data['UnitPrice'] = data['UnitPrice'].astype(float)

In [9]:
data['CustomerID'] = data['CustomerID'].astype(int)

Также приведем значения столбца InvoiceDate из объекта во время. И далее в удобное для анализа отображение даты.

In [10]:
data['InvoiceDate'] = data['InvoiceDate'].apply(lambda x: x.replace('/', '-'))
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12-1-10 8:26,3.39,17850,United Kingdom
1,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12-1-10 8:34,1.69,13047,United Kingdom
2,536367,21755,LOVE BUILDING BLOCK WORD,3,12-1-10 8:34,5.95,13047,United Kingdom
3,536368,22914,BLUE COAT RACK PARIS FASHION,3,12-1-10 8:34,4.95,13047,United Kingdom
4,536373,84406B,CREAM CUPID HEARTS COAT HANGER,8,12-1-10 9:02,2.75,17850,United Kingdom


In [11]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'], format = '%m-%d-%y %H:%M')
data.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
1,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047,United Kingdom
2,536367,21755,LOVE BUILDING BLOCK WORD,3,2010-12-01 08:34:00,5.95,13047,United Kingdom
3,536368,22914,BLUE COAT RACK PARIS FASHION,3,2010-12-01 08:34:00,4.95,13047,United Kingdom
4,536373,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 09:02:00,2.75,17850,United Kingdom
5,536375,82494L,WOODEN FRAME ANTIQUE WHITE,6,2010-12-01 09:32:00,2.55,17850,United Kingdom
6,536378,21033,JUMBO BAG CHARLIE AND LOLA TOYS,10,2010-12-01 09:37:00,2.95,14688,United Kingdom
7,536381,82567,"AIRLINE LOUNGE,METAL SIGN",2,2010-12-01 09:41:00,2.10,15311,United Kingdom
8,536381,21169,YOU'RE CONFUSING ME METAL SIGN,3,2010-12-01 09:41:00,1.69,15311,United Kingdom
9,536381,21175,GIN + TONIC DIET METAL SIGN,2,2010-12-01 09:41:00,2.10,15311,United Kingdom


In [12]:
data['InvoiceDate'] = data['InvoiceDate'].apply(lambda x: x.strftime(format = '%Y-%m-%d'))
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01,3.39,17850,United Kingdom
1,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01,1.69,13047,United Kingdom
2,536367,21755,LOVE BUILDING BLOCK WORD,3,2010-12-01,5.95,13047,United Kingdom
3,536368,22914,BLUE COAT RACK PARIS FASHION,3,2010-12-01,4.95,13047,United Kingdom
4,536373,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01,2.75,17850,United Kingdom


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40643 entries, 0 to 54213
Data columns (total 8 columns):
InvoiceNo      40643 non-null object
StockCode      40643 non-null object
Description    40643 non-null object
Quantity       40643 non-null int64
InvoiceDate    40643 non-null object
UnitPrice      40643 non-null float64
CustomerID     40643 non-null int32
Country        40643 non-null object
dtypes: float64(1), int32(1), int64(1), object(5)
memory usage: 2.6+ MB


In [14]:
data.describe()

,Quantity,UnitPrice,CustomerID
count,40643.000000,40643.000000,40643.000000
mean,12.199050,3.474702,15283.422803
std,43.583236,38.124841,1707.982278
min,-1296.000000,0.000000,12347.000000
25%,2.000000,1.250000,13969.000000
50%,5.000000,1.950000,15152.000000
75%,12.000000,3.750000,16775.000000
max,2400.000000,6930.000000,18287.000000


Также дата сет содержит отрицательные значения в столбце Количество, я предполагаю, что это означает - возврат товара. Общее количество таких строк = 921. Такие счета фактуры имеют приставку С

In [15]:
data[data['Quantity']<0].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
19,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01,4.65,15311,United Kingdom
30,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01,0.29,17548,United Kingdom
31,C536391,21980,PACK OF 12 RED RETROSPOT TISSUES,-24,2010-12-01,0.29,17548,United Kingdom
202,C536548,20957,PORCELAIN HANGING BELL SMALL,-1,2010-12-01,1.45,12472,Germany
375,C536642,21463,MIRRORED DISCO BALL,-1,2010-12-02,5.95,14390,United Kingdom


Посчитаем какое количество заказов совершают жители разных стран. Наиболее активными покупателями являются жители Великобритании.

In [16]:
data1 = data[data['Quantity']>0]
data1['Country'].value_counts()

United Kingdom          35372
Germany                   909
France                    859
EIRE                      699
Spain                     251
Netherlands               232
Belgium                   206
Switzerland               177
Portugal                  147
Australia                 123
Norway                    105
Channel Islands            81
Italy                      74
Finland                    72
Cyprus                     60
Sweden                     49
Austria                    48
Poland                     42
Denmark                    35
Japan                      27
Singapore                  24
Unspecified                21
Israel                     20
Iceland                    15
Greece                     14
Canada                     13
USA                        12
Malta                       8
Lithuania                   6
United Arab Emirates        5
European Community          5
RSA                         4
Brazil                      3
Czech Repu

Посчитаем выручку со всех транзакций

In [17]:
def revenue(a):
    """Подсчет конверсии переходов в покупки"""
    return a['Quantity'] * a['UnitPrice']
data['revenue'] = data.apply(revenue, axis=1)
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,revenue
0,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01,3.39,17850,United Kingdom,20.34
1,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01,1.69,13047,United Kingdom,54.08
2,536367,21755,LOVE BUILDING BLOCK WORD,3,2010-12-01,5.95,13047,United Kingdom,17.85
3,536368,22914,BLUE COAT RACK PARIS FASHION,3,2010-12-01,4.95,13047,United Kingdom,14.85
4,536373,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01,2.75,17850,United Kingdom,22.00


In [18]:
data['revenue'].sum()

840206.2109999999

Расчитаем LTV

In [19]:
data[['CustomerID', 'Country', 'revenue']].groupby('CustomerID').sum().sort_values(by='revenue', ascending=False).reset_index()

,CustomerID,revenue
0,18102,26533.44
1,14646,25312.76
2,17450,18233.46
3,12415,16030.81
4,14911,14674.63
5,17949,12048.08
6,14156,11775.91
7,17511,8961.03
8,14096,7974.99
9,15769,7914.24


Мы знаем сумму LTV, выведем на дашборд значения вместе со страной, чтобы понимать в какиз странах, помимо Великобритании есть потенциал в увеличении средств на рекламу. 
- Гипотеза - в других странах есть платежеспособные клиенты, но из-за слабой рекламной компании охват аудитории низкий. По этому мы увеличиваем рекламный бюджет в конкретных странах, и транслируем ее на популярных ресурсах данных стран.
- Проверять гипотезу будем путем сравнения с предыдущими месяцами: 1. количества уникальных клиентов в данных странах, 2.  Количество заказов клиентами. 3. Доход.

Мы видим, что по некоторым клиентам сумма дохода отрицательна, означает ли это, что клиенты делали только возвраты? Проверим

In [20]:
data[data['CustomerID'] == 16029.0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,revenue
22,536386,85099C,JUMBO BAG BAROQUE BLACK WHITE,100,2010-12-01,1.65,16029,United Kingdom,165.00
3242,539101,22466,FAIRY TALE COTTAGE NIGHTLIGHT,408,2010-12-16,1.45,16029,United Kingdom,591.60
3243,539101,21731,RED TOADSTOOL LED NIGHT LIGHT,408,2010-12-16,1.25,16029,United Kingdom,510.00
4337,540029,22207,FRYING PAN UNION FLAG,24,2011-01-04,3.75,16029,United Kingdom,90.00
6498,541611,79321,CHILLI LIGHTS,96,2011-01-20,3.82,16029,United Kingdom,366.72
12254,546734,22932,BAKING MOULD TOFFEE CUP CHOCOLATE,96,2011-03-16,2.10,16029,United Kingdom,201.60
15362,549594,22779,WOODEN OWLS LIGHT GARLAND,48,2011-04-11,3.37,16029,United Kingdom,161.76
17425,C551699,M,Manual,-1,2011-05-03,6930.00,16029,United Kingdom,-6930.00
18605,552786,22779,WOODEN OWLS LIGHT GARLAND,48,2011-05-11,3.37,16029,United Kingdom,161.76
21544,555585,22386,JUMBO BAG PINK POLKADOT,100,2011-06-06,1.74,16029,United Kingdom,174.00


Данный клиент делал как покупки так и возвраты, почему стоимость Руководства по эксклуатации составляет 6 930 фунтов и почему его вернули, возможно это техническая ошибка при сборе данных.

Рассчитаем доход с платящего клиента

In [21]:
data1 = data[data['Quantity']>0]
arppu = data1['revenue'].sum()/data1['CustomerID'].count()
arppu

22.21168548914959

Рассчитаем доход, получаемый от клиентов из разных стран

In [22]:
data1[['Country', 'revenue']].groupby('Country').sum().sort_values(by='revenue', ascending=False)

,revenue
Country,
United Kingdom,720938.721
EIRE,27733.900
Netherlands,26489.180
Germany,23706.160
France,19758.170
Australia,17156.590
Switzerland,6144.250
Spain,5971.220
Sweden,4936.520


Аналогично рассчитаем убыток

In [23]:
data2 = data[data['Quantity']<0]
data2[['Country', 'revenue']].groupby('Country').sum().sort_values(by='revenue', ascending=True)

,revenue
Country,
United Kingdom,-36688.82
Sweden,-1188.00
Germany,-1186.96
EIRE,-880.62
Portugal,-557.72
France,-338.49
Cyprus,-320.69
Netherlands,-221.40
USA,-188.68


Датасет отчищен от пустых значений, данные преобразованы для работы и вывода в дашборд. 